In [16]:
from dotenv import load_dotenv
import requests
requests.packages.urllib3.disable_warnings()
import pandas as pd
import json
import os

load_dotenv()

url = os.getenv("FANDUEL_URL")
token = os.getenv("FANDUEL_TOKEN")
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
           "Authorization": f"Bearer {token}" 
}

with open ('cookie.json', 'r') as cookie_file:
    cookie = json.load(cookie_file)

session = requests.Session()

r = requests.get(url, headers=headers, cookies=cookie, verify=False).json()


Now that we have the latest odds from fanduel, we need to traverse the JSON structure to find the market we need.

We will focus on the alternative runline market.

We will get the data from Fanduel and transform it into a Pandas dataframe

In [17]:
markets = [x for x in r['attachments']['markets']]

data = []

for market in markets:
    x = r['attachments']['markets'][market]

    if x['marketName'] == "Total Runs":
        event_id = x['eventId']
        event_name = r['attachments']['events'][str(event_id)]['name']
        
        under = [x for x in x['runners'] if x['runnerName']=='Under'][0]
        under_odds = {"type": under['runnerName'],
                      "handicap": under['handicap'],
                      "odds": under['winRunnerOdds']['americanDisplayOdds']['americanOdds']}
        
        over = [x for x in x['runners'] if x['runnerName']=='Over'][0]
        over_odds = {"type": over['runnerName'],
                     "handicap": over['handicap'],
                     "odds": over['winRunnerOdds']['americanDisplayOdds']['americanOdds']}
        
        event_data = {
            "event_name": event_name,
            "event_id": event_id,
            "type": "total runs",
            "under_handicap": under_odds['handicap'],
            "under_odds": under_odds['odds'],
            "over_handicap": over_odds['handicap'],
            "over_odds": over_odds['odds'] 
        }

        data.append(event_data)

Now we can work with the data in Pandas

In [18]:
tr_df = pd.DataFrame.from_records(data=data)
print(tr_df)

                                           event_name  event_id        type  \
0   Pittsburgh Pirates (J Jones) @ New York Yankee...  33632085  total runs   
1   Philadelphia Phillies (Z Wheeler) @ Washington...  33635510  total runs   
2   Los Angeles Dodgers (Y Yamamoto) @ Colorado Ro...  33635515  total runs   
3   Cincinnati Reds (R Lowder) @ Chicago Cubs (K H...  33635509  total runs   
4   Houston Astros (R Blanco) @ Cleveland Guardian...  33632079  total runs   
5   Oakland Athletics (J Sears) @ Seattle Mariners...  33632084  total runs   
6   St. Louis Cardinals (M Mikolas) @ San Francisc...  33632078  total runs   
7   Baltimore Orioles (C Povich) @ Minnesota Twins...  33632082  total runs   
8   Miami Marlins (A Oller) @ Toronto Blue Jays (J...  33632086  total runs   
9   Philadelphia Phillies (R Suarez) @ Washington ...  33632075  total runs   
10  Los Angeles Dodgers (R Brasier) @ Colorado Roc...  33632077  total runs   
11  St. Louis Cardinals (A Pallante) @ San Francis..